In [1]:
import pickle 
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model

In [2]:
## Load the ANN trained model, Scaler pickle file, onehot encoder pickle file and label encoder pickle file
model = load_model('model.h5')

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('onehot_encoder_geo.pkl','rb') as file:
    onehot_encoder_geo = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [3]:
## Example Input data
input_data = {
    'CreditScore' : 600,
    'Geography' : 'France',
    'Gender' : 'Male',
	'Age' : 40,
	'Tenure' : 3,
	'Balance' : 60000,
	'NumOfProducts' : 2,
	'HasCrCard' : 1,
    'IsActiveMember' : 1,
	'EstimatedSalary' : 50000
}

In [4]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [5]:
## Label encode 'Gender'
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [6]:
## One hot encode 'Geography'
geo_encoded = onehot_encoder_geo.transform(input_df[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [7]:
## Concatenate with original input DataFrame and drop original 'Geography' column
input_df = pd.concat([input_df.drop('Geography', axis=1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [8]:
## Scaling the input data
input_df = scaler.transform(input_df)
input_df

array([[-0.54007722,  0.91186722,  0.10685843, -0.69164931, -0.26034447,
         0.81311138,  0.64946204,  0.96978946, -0.88017893,  1.00053348,
        -0.57776083, -0.57735027]])

In [9]:
## Predict Churn
prediction = model.predict(input_df)
prediction_proba = prediction[0][0]
print(prediction)
print(prediction_proba)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
[[0.05360068]]
0.053600676


In [10]:
if prediction_proba > 0.5 :
    print("The customer id likely to churn.")
else : 
    print("The customer is not likely to churn.")

The customer is not likely to churn.
